In [1]:
import argparse
import os

import apache_beam as beam
import tensorflow as tf
from apache_beam.options.pipeline_options import PipelineOptions
import apache_beam.runners.interactive.interactive_beam as ib
import apache_beam.transforms.sql

import beam__common
import fidscs_globals
import random

from importlib import import_module
data_extractor = import_module('data-extractor', '.')

In [2]:
data_dir = "/tmp/fids-capstone-data/data"

data_extractor.run(max_target_videos=-1, data_dir=data_dir, use_beam=True)

use_beam: True
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CollectiveCommunication.AUTO
Number of devices available for parallel processing: 1
PipelineOptions:
{'runner': 'DirectRunner', 'streaming': False, 'beam_services': {}, 'type_check_strictness': 'DEFAULT_TO_ANY', 'type_check_additional': '', 'pipeline_type_check': True, 'runtime_type_check': False, 'performance_runtime_type_check': False, 'direct_runner_use_stacked_bundle': True, 'direct_runner_bundle_repeat': 0, 'direct_num_workers': 0, 'direct_running_mode': 'multi_threading', 'dataflow_endpoint': 'https://dataflow.googleapis.com', 'project': 'my-project', 'job_name': None, 'staging_location': None, 'temp_location': None, 'region': None, 'service_account_email': None, 'no_auth': False, 'template_location': None, 'labels': None, 'update': False, 'transform_name_mapping': None, 'enable_str

In [3]:
options = {
    'project': 'my-project', # change
    # 'runner': 'DirectRunner',
    'runner': 'InteractiveRunner',
    'direct_num_workers': 0, # 0 is use all available cores
    'direct_running_mode': 'multi_threading', # ['in_memory', 'multi_threading', 'multi_processing'] # 'multi_processing' doesn't seem to work for DirectRunner?
    'streaming': False # set to True if data source is unbounded (e.g. GCP PubSub)
}
pipeline_options = PipelineOptions(flags=[], **options) # easier to pass in options from command-line this way
print(f"PipelineOptions:\n{pipeline_options.get_all_options()}\n")

PipelineOptions:
{'runner': 'InteractiveRunner', 'streaming': False, 'beam_services': {}, 'type_check_strictness': 'DEFAULT_TO_ANY', 'type_check_additional': '', 'pipeline_type_check': True, 'runtime_type_check': False, 'performance_runtime_type_check': False, 'direct_runner_use_stacked_bundle': True, 'direct_runner_bundle_repeat': 0, 'direct_num_workers': 0, 'direct_running_mode': 'multi_threading', 'dataflow_endpoint': 'https://dataflow.googleapis.com', 'project': 'my-project', 'job_name': None, 'staging_location': None, 'temp_location': None, 'region': None, 'service_account_email': None, 'no_auth': False, 'template_location': None, 'labels': None, 'update': False, 'transform_name_mapping': None, 'enable_streaming_engine': False, 'dataflow_kms_key': None, 'flexrs_goal': None, 'hdfs_host': None, 'hdfs_port': None, 'hdfs_user': None, 'hdfs_full_urls': False, 'num_workers': None, 'max_num_workers': None, 'autoscaling_algorithm': None, 'machine_type': None, 'disk_size_gb': None, 'disk_t

In [4]:
fidscs_globals.DATA_ROOT_DIR = data_dir

In [5]:
can_proceed = True

if not tf.io.gfile.exists(fidscs_globals.DATA_ROOT_DIR) or len(tf.io.gfile.listdir(fidscs_globals.DATA_ROOT_DIR))==0:
    print(f"{fidscs_globals.VALIDATION_FATAL_ERROR_TEXT} data directory does not exist or is empty!")
    can_proceed = False
else:
    fidscs_globals.VIDEO_DIR = os.path.join(fidscs_globals.DATA_ROOT_DIR, 'videos')
    fidscs_globals.STICHED_VIDEO_FRAMES_DIR = os.path.join(fidscs_globals.DATA_ROOT_DIR, 'stitched_video_frames')
    fidscs_globals.CORPUS_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.CORPUS_DS_FNAME)
    fidscs_globals.DOCUMENT_ASL_CONSULTANT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.DOCUMENT_ASL_CONSULTANT_DS_FNAME)
    fidscs_globals.ASL_CONSULTANT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.ASL_CONSULTANT_DS_FNAME)
    fidscs_globals.VIDEO_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_DS_FNAME)
    fidscs_globals.VIDEO_SEGMENT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_SEGMENT_DS_FNAME)
    fidscs_globals.VIDEO_FRAME_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_FRAME_DS_FNAME)
    fidscs_globals.UTTERANCE_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_DS_FNAME)
    fidscs_globals.UTTERANCE_VIDEO_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_VIDEO_DS_FNAME)
    fidscs_globals.UTTERANCE_TOKEN_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_TOKEN_DS_FNAME)
    fidscs_globals.UTTERANCE_TOKEN_FRAME_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_TOKEN_FRAME_DS_FNAME)
    fidscs_globals.VOCABULARY_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VOCABULARY_DS_FNAME)

In [6]:
pl = beam.Pipeline(options=pipeline_options)

full_target_vid_index_schemad_pcoll = beam__common.pl__1__read_target_vid_index_csv(pl)
corpus_index_schemad_pcoll = beam__common.pl__1__read_corpus_index_csv(pl) # XML is base-64 encode but we no longer need it (to decode it) since it is only used to create the datasets
# corpus_index_decoded_XML_pcoll = pl__2__decode_XML(corpus_index_schemad_pcoll) # see above

asl_consultant_index_schemad_pcoll = beam__common.pl__1__read_asl_consultant_index_csv(pl)
document_asl_consultant_utterance_index_schemad_pcoll = beam__common.pl__1__read_document_asl_consultant_utterance_index_csv(pl)
document_asl_consultant_target_video_index_schemad_pcoll = beam__common.pl__1__read_document_asl_consultant_target_video_index_csv(pl)
document_asl_consultant_utterance_video_index_schemad_pcoll = beam__common.pl__1__read_document_asl_consultant_utterance_video_index_csv(pl)
document_target_video_segment_index_schemad_pcoll = beam__common.pl__1__read_document_target_video_segment_index_csv(pl)
vocabulary_index_schemad_pcoll = beam__common.pl__1__read_vocabulary_index_csv(pl)
document_asl_consultant_utterance_token_index_schemad_pcoll = beam__common.pl__1__read_document_asl_consultant_utterance_token_index_csv(pl)
document_asl_consultant_target_video_frame_index_schemad_pcoll = beam__common.pl__1__read_document_asl_consultant_target_video_frame_index_csv(pl)
document_asl_consultant_target_video_utterance_token_frame_index_schemad_pcoll = beam__common.pl__1__read_document_asl_consultant_target_video_utterance_token_frame_index_csv(pl)

In [7]:
# document_asl_consultant_target_video_utterance_token_frame_index_schemad_pcoll is the main table we use for training.
#     This will ultimately provide which frame sequences correspond to individual tokens.

# But our first measure is to build train and validation sets (for tokens).
#   In order to split up train vs validation sets, we need to compare "apples to apples".
#   That is, in order for a token (TokenID) to be considered a candidate for the split,
#   we require at least two of the same (TokenID, CameraPerspective) wherein the ASL
#   consultant for each differs.  We would prefer more than two of these tuples, each
#   having unique ASL consultants in the set of occurrences, with the majority of said
#   tuples being assigned to the training set and the remainder (at least one) being
#   assigned to the validation set.  We would like to achieve a 90/10 split, ideally,
#   but we will take what we get.

# document_asl_consultant_target_video_utterance_token_frame_index_schemad_pcoll:
    # beam.Row(
    #   DocumentID=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[0]]),
    #   ASLConsultantID=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[1]]),
    #   CameraPerspective=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[2]]),
    #   TargetVideoFilename=str(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[3]]),
    #   UtteranceSequence=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[4]]),
    #   TokenSequence=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[5]]),
    #   FrameSequence=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[6]]),
    #   TokenID=int(d_document_asl_consultant_target_video_utterance_token_frame_info[fidscs_globals.SCHEMA_COL_NAMES__UTTERANCE_TOKEN_FRAME_DS[7]])
    # )

# We will transform this into tuples of the form:
    # [
    #     'TokenID', 
    #     'CameraPerspective', 
    #     'DocumentID', 
    #     'ASLConsultantID', 
    #     'TargetVideoFilename', 
    #     'UtteranceSequence', 
    #     'TokenSequence',

    #     'FrameSequence'
    # ]

dctvustsfs = (
    document_asl_consultant_target_video_utterance_token_frame_index_schemad_pcoll
    | "Beam PL: extract (TokenID,CameraPerspective,ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence,FrameSequence) from dctvustsfs schemad pcoll" >> beam.Map(
            lambda dctvustsfs_row: (
                dctvustsfs_row.TokenID,
                dctvustsfs_row.CameraPerspective,
                dctvustsfs_row.ASLConsultantID,
                dctvustsfs_row.TargetVideoFilename,
                dctvustsfs_row.UtteranceSequence,
                dctvustsfs_row.TokenSequence,
                dctvustsfs_row.FrameSequence
            )
        )
)

# for train-validation split, we want to key/group by (TokenID, CameraPerspective) with lists of unique (ASLConsultantID, TargetVideoFilename, UtteranceSequence, TokenSequence) > 1
ctvusts_by_tcp = (
    dctvustsfs
    | "Beam PL: extract ((TokenID,CameraPerspective), (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence)) from dctvustsfs" >> beam.Map(
            lambda dctvustsfs_row_tpl: (
                (
                    dctvustsfs_row_tpl[0],
                    dctvustsfs_row_tpl[1]
                ),
                (
                    dctvustsfs_row_tpl[2],
                    dctvustsfs_row_tpl[3],
                    dctvustsfs_row_tpl[4],
                    dctvustsfs_row_tpl[5]
                )
            )
        )
    | "Beam PL: select distinct ((TokenID,CameraPerspective), (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence)) from ctvusts_by_tcp" >> beam.Distinct()
    | "Beam PL: group (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence) by key (TokenID,CameraPerspective)" >> beam.GroupByKey() 
    # the above produces tuples of the form:
        # (
        #     (
        #         TokenID,
        #         CameraPerspective
        #     ),
        #     listof(
        #       (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence)
        #     )
        # )
)


def flatten_ctvusts_by_tcp(ctvusts_by_tcp_tpl):
    return [
        (
            ctvusts_by_tcp_tpl[0][0],   # TokenID
            ctvusts_by_tcp_tpl[0][1],   # CameraPerspective
            ctvusts_tpl[0],             # ASLConsultantID
            ctvusts_tpl[1],             # TargetVideoFilename
            ctvusts_tpl[2],             # UtteranceSequence
            ctvusts_tpl[3]              # TokenSequence
        ) for ctvusts_tpl in ctvusts_by_tcp_tpl[1]
    ]

ctvusts_by_tcp__gt_1 = (
    ctvusts_by_tcp
    | "Beam PL: filter candidate (TokenID,CameraPerspective) for test-validation split" >> beam.Filter(
            lambda list_ctvusts_by_tcp_tpl: len(set(list_ctvusts_by_tcp_tpl[1])) > 1
        )
    | "Beam PL: flatten filtered (TokenID,CameraPerspective) candidates for test-validation split" >> beam.FlatMap(flatten_ctvusts_by_tcp)
)

ctvusts_by_tcp__lte_1 = (
    ctvusts_by_tcp
    | "Beam PL: filter non-candidate (TokenID,CameraPerspective) for test-validation split" >> beam.Filter(
            lambda list_ctvusts_by_tcp_tpl: len(set(list_ctvusts_by_tcp_tpl[1])) <= 1
        )
    | "Beam PL: flatten filtered (TokenID,CameraPerspective) non-candidates for test-validation split" >> beam.FlatMap(flatten_ctvusts_by_tcp)
)

<p><br>

#### Finally, execute validation/train split on ctvusts_by_tcp__gt_1

In [8]:
# first, we need to put ctvusts_by_tcp__gt_1 back into ((TokenID, CameraPerspective), (ASLConsultantID, TargetVideoFilename, UtteranceSequence, TokenSequence)) form
def rekey_ctvusts_by_tcp(ctvusts_by_tcp_tpl):
    return (
        (
            ctvusts_by_tcp_tpl[0],  # TokenID
            ctvusts_by_tcp_tpl[1]   # CameraPerspective
        ),
        (
            ctvusts_by_tcp_tpl[2],  # ASLConsultantID
            ctvusts_by_tcp_tpl[3],  # TargetVideoFilename
            ctvusts_by_tcp_tpl[4],  # UtteranceSequence
            ctvusts_by_tcp_tpl[5]   # TokenSequence
        )
    )

def val_train_split__ctvusts_by_tcp__gt_1__tpl(ctvusts_list__by__tcp__gt_1__tpl):
    """
    ctvusts_list__by__tcp__gt_1__tpl
        (
            (TokenID,CameraPerspective), # key
            listof(
                (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence)
            )
        )
    """
    ctvusts_list = ctvusts_list__by__tcp__gt_1__tpl[1].copy() # we need a copy since we want to shuffle
    random.shuffle(ctvusts_list)
    len_ctvusts_list = len(ctvusts_list)
    val_len_ctvusts_list = int(len_ctvusts_list*fidscs_globals.VALIDATION_SIZE_RATIO) if len_ctvusts_list > int(((1-fidscs_globals.VALIDATION_SIZE_RATIO)*100)/10) else 1
    train__ctvusts_list, val__ctvusts_list = ctvusts_list[val_len_ctvusts_list:], ctvusts_list[:val_len_ctvusts_list]
    return (
        (
            ctvusts_list__by__tcp__gt_1__tpl[0][0],    # TokenID
            ctvusts_list__by__tcp__gt_1__tpl[0][1]     # CameraPerspective
        ),
        (
            train__ctvusts_list,
            val__ctvusts_list
        )
    )

val_train_split_basis__ctvusts_by_tcp__gt_1 = (
    ctvusts_by_tcp__gt_1
    | "Beam PL: rekey ctvusts_by_tcp__gt_1 for validation/train split" >> beam.Map(rekey_ctvusts_by_tcp)
    | "Beam PL: group (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence) rekeyed by (TokenID,CameraPerspective)" >> beam.GroupByKey()
    # the above produces tuples of the form:
        # (
        #     (TokenID,CameraPerspective), # key
        #     listof(
        #       (ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence)
        #     )
        # )
    | "Beam PL: split rekeyed ctvusts_list_by_tcp__gt_1" >> beam.Map(val_train_split__ctvusts_by_tcp__gt_1__tpl)
    # the above produces tuples of the form:
        # (
        #     (TokenID,CameraPerspective), # key
        #     (
        #       test_list_of(ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence),
        #       val_list_of(ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence),
        #     )
        # )
)

train__ctvusts_by_tcp__gt_1 = (
    val_train_split_basis__ctvusts_by_tcp__gt_1
    | "Beam PL: select train sublist from val_train_split_basis__ctvusts_by_tcp__gt_1" >> beam.Map(
            lambda val_train_split_basis__ctvusts_by_tcp__gt_1_tpl: [
                (
                    val_train_split_basis__ctvusts_by_tcp__gt_1_tpl[0][0],  # TokenID
                    val_train_split_basis__ctvusts_by_tcp__gt_1_tpl[0][1],  # CameraPerspective
                    train_ctvusts_tpl[0],                                   # ASLConsultantID
                    train_ctvusts_tpl[1],                                   # TargetVideoFilename
                    train_ctvusts_tpl[2],                                   # UtteranceSequence
                    train_ctvusts_tpl[3],                                   # UtteranceSequence
                ) for train_ctvusts_tpl in val_train_split_basis__ctvusts_by_tcp__gt_1_tpl[1][0] # index [1][0] points to train sublist
            ]
        )
    | "Beam PL: 'explode list_train__ctvusts_by_tcp__gt_1_tpl" >> beam.FlatMap(lambda list_train__ctvusts_by_tcp__gt_1_tpl: list_train__ctvusts_by_tcp__gt_1_tpl)
)

val__ctvusts_by_tcp__gt_1 = (
    val_train_split_basis__ctvusts_by_tcp__gt_1
    | "Beam PL: select validation sublist from val_train_split_basis__ctvusts_by_tcp__gt_1" >> beam.Map(
            lambda val_train_split_basis__ctvusts_by_tcp__gt_1_tpl: [
                (
                    val_train_split_basis__ctvusts_by_tcp__gt_1_tpl[0][0],  # TokenID
                    val_train_split_basis__ctvusts_by_tcp__gt_1_tpl[0][1],  # CameraPerspective
                    val_ctvusts_tpl[0],                                     # ASLConsultantID
                    val_ctvusts_tpl[1],                                     # TargetVideoFilename
                    val_ctvusts_tpl[2],                                     # UtteranceSequence
                    val_ctvusts_tpl[3],                                     # UtteranceSequence
                ) for val_ctvusts_tpl in val_train_split_basis__ctvusts_by_tcp__gt_1_tpl[1][1] # index [1][1] points to validation sublist
            ]
        )
    | "Beam PL: 'explode list_val__ctvusts_by_tcp__gt_1_tpl" >> beam.FlatMap(lambda list_val__ctvusts_by_tcp__gt_1_tpl: list_val__ctvusts_by_tcp__gt_1_tpl)
)

# train_supp__ctvusts_by_tcp__lte_1 = (
#     ctvusts_by_tcp__lte_1
#     | "Beam PL: rekey ctvusts_by_tcp__lte_1 for supplemental train" >> beam.Map(rekey_ctvusts_by_tcp)
# )

In [9]:
# we require this in order to make use of ib.show() (which provides visualization of the pcolls specified) or ib.collect() (which creates a pandas dataframe from a pcoll)
    # but all pcolls we wish to visualize must be created prior to executing the following line
ib.watch(locals())

#### Show those with counts > 1

In [10]:
df_ctvusts_by_tcp__gt_1 = ib.collect(ctvusts_by_tcp__gt_1)

In [11]:
df_ctvusts_by_tcp__gt_1.columns = ['TokenID', 'CameraPerspective', 'ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence']
df_ctvusts_by_tcp__gt_1.set_index(['TokenID', 'CameraPerspective'], inplace=True)
df_ctvusts_by_tcp__gt_1.sort_values(axis=0, by=['ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence'], ignore_index=False, inplace=True)
df_ctvusts_by_tcp__gt_1.sort_index(inplace=True)
df_ctvusts_by_tcp__gt_1

ASLConsultantID   TargetVideoFilename  \
TokenID CameraPerspective                                          
0       0                                1   DSP%2520DeadDog.mov   
        0                                3  d18_1149_small_0.mov   
        0                                4     _1337_small_0.mov   
        0                                4     _1339_small_0.mov   
        0                                4     _1339_small_0.mov   
...                                    ...                   ...   
2396    0                                4     _1449_small_0.mov   
        1                                4     _1441_small_1.mov   
        1                                4     _1449_small_1.mov   
        3                                4     _1441_small_3.mov   
        3                                4     _1449_small_3.mov   

                           UtteranceSequence  TokenSequence  
TokenID CameraPerspective                                    
0       0                                 11              2  
        0                                  5              0  
        0                                 42              0  
        0                                 44              0  
        0                                 44              4  
...                                      ...            ...  
2396    0                                151              4  
        1                                143              5  
        1                                151              4  
        3                                143              5  
        3                                151              4  

[29762 rows x 4 columns]

In [12]:
# df_ctvusts_by_tcp__gt_1.loc[
#     (
#         [2369],         # TokenID
#         [2]             # CameraPerspective
#     ), 
#     :
# ].sort_index(ascending=[True, True])

In [13]:
df_ctvusts_by_tcp__gt_1__count = df_ctvusts_by_tcp__gt_1.reset_index().groupby(['TokenID', 'CameraPerspective']).count()
df_ctvusts_by_tcp__gt_1__count = df_ctvusts_by_tcp__gt_1__count[['ASLConsultantID']]
df_ctvusts_by_tcp__gt_1__count.columns = ['count']
df_ctvusts_by_tcp__gt_1__count.sort_values(axis=0, by=['count'], ascending=False, inplace=True)
# df_ctvusts_by_tcp__gt_1__count.sort_index(inplace=True)
df_ctvusts_by_tcp__gt_1__count

count
TokenID CameraPerspective       
906     0                    694
0       0                    629
906     2                    570
0       2                    559
318     0                    414
...                          ...
316     3                      2
        2                      2
        0                      2
588     0                      2
2396    3                      2

[3079 rows x 1 columns]

#### Now show those with counts <= 1

In [14]:
df_ctvusts_by_tcp__lte_1 = ib.collect(ctvusts_by_tcp__lte_1)

In [15]:
df_ctvusts_by_tcp__lte_1.columns = ['TokenID', 'CameraPerspective', 'ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence']
df_ctvusts_by_tcp__lte_1.set_index(['TokenID', 'CameraPerspective'], inplace=True)
df_ctvusts_by_tcp__lte_1.sort_values(axis=0, by=['ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence'], ignore_index=False, inplace=True)
df_ctvusts_by_tcp__lte_1.sort_index(inplace=True)
df_ctvusts_by_tcp__lte_1

ASLConsultantID           TargetVideoFilename  \
TokenID CameraPerspective                                                  
1       0                                6     ben_story_439_small_0.mov   
        1                                6     ben_story_439_small_1.mov   
        2                                6     ben_story_439_small_2.mov   
        3                                6     ben_story_439_small_3.mov   
2       0                                6     ben_story_439_small_0.mov   
...                                    ...                           ...   
2409    0                                0        DSP%2520Immigrants.mov   
2410    0                                4    boston-la_1088_small_0.mov   
        2                                4    boston-la_1088_small_2.mov   
2411    0                                4  biker_buddy_1069_small_0.mov   
        2                                4  biker_buddy_1069_small_2.mov   

                           UtteranceSequence  TokenSequence  
TokenID CameraPerspective                                    
1       0                                 27              4  
        1                                 27              4  
        2                                 27              4  
        3                                 27              4  
2       0                                 45              3  
...                                      ...            ...  
2409    0                                 15              4  
2410    0                                 35              3  
        2                                 35              3  
2411    0                                  9              3  
        2                                  9              3  

[3691 rows x 4 columns]

In [16]:
df_ctvusts_by_tcp__lte_1__count = df_ctvusts_by_tcp__lte_1.reset_index().groupby(['TokenID', 'CameraPerspective']).count()
df_ctvusts_by_tcp__lte_1__count = df_ctvusts_by_tcp__lte_1__count[['ASLConsultantID']]
df_ctvusts_by_tcp__lte_1__count.columns = ['count']
df_ctvusts_by_tcp__lte_1__count.sort_values(axis=0, by=['count'], ascending=False, inplace=True)
# df_ctvusts_by_tcp__gt_1__count.sort_index(inplace=True)
df_ctvusts_by_tcp__lte_1__count

count
TokenID CameraPerspective       
1       0                      1
1621    0                      1
1626    0                      1
        2                      1
1631    0                      1
...                          ...
828     0                      1
        2                      1
829     0                      1
        2                      1
2411    2                      1

[3691 rows x 1 columns]

In [17]:
df_ctvusts_by_tcp__intersection = df_ctvusts_by_tcp__gt_1.join(df_ctvusts_by_tcp__lte_1, how='inner', lsuffix='_left', rsuffix='_right')
df_ctvusts_by_tcp__intersection

,,ASLConsultantID_left,TargetVideoFilename_left,UtteranceSequence_left,TokenSequence_left,ASLConsultantID_right,TargetVideoFilename_right,UtteranceSequence_right,TokenSequence_right
TokenID,CameraPerspective,,,,,,,,


#### Now show train/validation split

In [18]:
df_train__ctvusts_by_tcp__gt_1 = ib.collect(train__ctvusts_by_tcp__gt_1)

In [20]:
df_train__ctvusts_by_tcp__gt_1.columns = ['TokenID', 'CameraPerspective', 'ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence']
df_train__ctvusts_by_tcp__gt_1.set_index(['TokenID', 'CameraPerspective'], inplace=True)
df_train__ctvusts_by_tcp__gt_1.sort_values(axis=0, by=['ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence'], ignore_index=False, inplace=True)
df_train__ctvusts_by_tcp__gt_1.sort_index(inplace=True)
df_train__ctvusts_by_tcp__gt_1

ASLConsultantID          TargetVideoFilename  \
TokenID CameraPerspective                                                 
0       0                                1          DSP%2520DeadDog.mov   
        0                                3         d18_1149_small_0.mov   
        0                                4            _1337_small_0.mov   
        0                                4            _1339_small_0.mov   
        0                                4            _1344_small_0.mov   
...                                    ...                          ...   
2394    2                                4  whitewater_1049_small_2.mov   
        3                                4  whitewater_1049_small_3.mov   
2396    0                                4            _1449_small_0.mov   
        1                                4            _1441_small_1.mov   
        3                                4            _1449_small_3.mov   

                           UtteranceSequence  TokenSequence  
TokenID CameraPerspective                                    
0       0                                 11              2  
        0                                  5              0  
        0                                 42              0  
        0                                 44              0  
        0                                 48              0  
...                                      ...            ...  
2394    2                                 12              2  
        3                                 40              4  
2396    0                                151              4  
        1                                143              5  
        3                                151              4  

[25381 rows x 4 columns]

In [21]:
df_val__ctvusts_by_tcp__gt_1 = ib.collect(val__ctvusts_by_tcp__gt_1)

In [22]:
df_val__ctvusts_by_tcp__gt_1.columns = ['TokenID', 'CameraPerspective', 'ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence']
df_val__ctvusts_by_tcp__gt_1.set_index(['TokenID', 'CameraPerspective'], inplace=True)
df_val__ctvusts_by_tcp__gt_1.sort_values(axis=0, by=['ASLConsultantID', 'TargetVideoFilename', 'UtteranceSequence', 'TokenSequence'], ignore_index=False, inplace=True)
df_val__ctvusts_by_tcp__gt_1.sort_index(inplace=True)
df_val__ctvusts_by_tcp__gt_1

ASLConsultantID          TargetVideoFilename  \
TokenID CameraPerspective                                                 
0       0                                4            _1339_small_0.mov   
        0                                4            _1365_small_0.mov   
        0                                4            _1400_small_0.mov   
        0                                4            _1442_small_0.mov   
        0                                4            _1445_small_0.mov   
...                                    ...                          ...   
2394    2                                4  whitewater_1049_small_2.mov   
        3                                4  whitewater_1049_small_3.mov   
2396    0                                4            _1441_small_0.mov   
        1                                4            _1449_small_1.mov   
        3                                4            _1441_small_3.mov   

                           UtteranceSequence  TokenSequence  
TokenID CameraPerspective                                    
0       0                                 44              4  
        0                                 69              0  
        0                                102              0  
        0                                144              1  
        0                                147              3  
...                                      ...            ...  
2394    2                                 40              4  
        3                                 12              2  
2396    0                                143              5  
        1                                151              4  
        3                                143              5  

[4381 rows x 4 columns]

In [23]:
df_train__ctvusts_by_tcp__gt_1.loc[
    (
        [2394],         # TokenID
        [2]             # CameraPerspective
    ), 
    :
].sort_index(ascending=[True, True])

,,ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence
TokenID,CameraPerspective,,,,
2394,2,4,whitewater_1049_small_2.mov,12,2


In [24]:
df_val__ctvusts_by_tcp__gt_1.loc[
    (
        [2394],         # TokenID
        [2]             # CameraPerspective
    ), 
    :
].sort_index(ascending=[True, True])

,,ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence
TokenID,CameraPerspective,,,,
2394,2,4,whitewater_1049_small_2.mov,40,4
